In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import glob
import os
import os
import cv2
import glob
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mlxtend.plotting import plot_confusion_matrix
from scipy import ndimage
import skimage
from skimage.io import imread
from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit as s_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop, Adam
from keras.layers import Conv2D , SeparableConv2D, BatchNormalization, UpSampling2D, MaxPool2D
from keras.layers import MaxPooling2D, Average, Input, Concatenate, LeakyReLU, Add, ELU, PReLU, ReLU
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras import backend as BE

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.datasets import cifar10

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\chandrachurh\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [4]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [5]:
X_train /= 255
X_test /= 255

In [6]:
BE.clear_session()

In [7]:
def resnet_model():
    base_model = ResNet50(weights = None, include_top = False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    pred = Dense(10, activation = "sigmoid")(x)
    model_resnet = Model(input=base_model.input, output = pred)
    return model_resnet

In [8]:
model_resnet=resnet_model()
model_resnet.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model_resnet.summary()

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
c:\users\chandrachurh\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

res3a_branch2a (Conv2D)         (None, None, None, 1 32896       activation_10[0][0]              
__________________________________________________________________________________________________
bn3a_branch2a (BatchNormalizati (None, None, None, 1 512         res3a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_11 (Activation)      (None, None, None, 1 0           bn3a_branch2a[0][0]              
__________________________________________________________________________________________________
res3a_branch2b (Conv2D)         (None, None, None, 1 147584      activation_11[0][0]              
__________________________________________________________________________________________________
bn3a_branch2b (BatchNormalizati (None, None, None, 1 512         res3a_branch2b[0][0]             
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
add_7 (Add)                     (None, None, None, 5 0           bn3d_branch2c[0][0]              
                                                                 activation_19[0][0]              
__________________________________________________________________________________________________
activation_22 (Activation)      (None, None, None, 5 0           add_7[0][0]                      
__________________________________________________________________________________________________
res4a_branch2a (Conv2D)         (None, None, None, 2 131328      activation_22[0][0]              
__________________________________________________________________________________________________
bn4a_branch2a (BatchNormalizati (None, None, None, 2 1024        res4a_branch2a[0][0]             
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
activation_33 (Activation)      (None, None, None, 2 0           bn4d_branch2b[0][0]              
__________________________________________________________________________________________________
res4d_branch2c (Conv2D)         (None, None, None, 1 263168      activation_33[0][0]              
__________________________________________________________________________________________________
bn4d_branch2c (BatchNormalizati (None, None, None, 1 4096        res4d_branch2c[0][0]             
__________________________________________________________________________________________________
add_11 (Add)                    (None, None, None, 1 0           bn4d_branch2c[0][0]              
                                                                 activation_31[0][0]              
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
activation_44 (Activation)      (None, None, None, 5 0           bn5b_branch2a[0][0]              
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, None, None, 5 2359808     activation_44[0][0]              
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, None, None, 5 2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
activation_45 (Activation)      (None, None, None, 5 0           bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2c (Conv2D)         (None, None, None, 2 1050624     activation_45[0][0]              
__________

In [9]:
nepoch = 100
batch_size = 32

#Applying early stopping
earlyStopping=EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')

lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=2, verbose=1)

save_path="weights_resnet_1.h5"
checkpoint = ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

c:\users\chandrachurh\appdata\local\programs\python\python35\lib\site-packages\keras\callbacks.py:1016: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [10]:
resnet_val_hist = model_resnet.fit(X_train, y_train, epochs = nepoch, batch_size=batch_size, callbacks = [lr_reduce, checkpoint, earlyStopping], validation_split = 0.1, verbose = 2)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
 - 313s - loss: 0.3279 - acc: 0.8987 - val_loss: 0.8608 - val_acc: 0.8980

Epoch 00001: val_acc improved from -inf to 0.89796, saving model to weights_resnet_1.h5
Epoch 2/100
 - 299s - loss: 0.3416 - acc: 0.8988 - val_loss: 0.4688 - val_acc: 0.9000

Epoch 00002: val_acc improved from 0.89796 to 0.90000, saving model to weights_resnet_1.h5
Epoch 3/100
 - 296s - loss: 0.3341 - acc: 0.8996 - val_loss: 0.2888 - val_acc: 0.9020

Epoch 00003: val_acc improved from 0.90000 to 0.90198, saving model to weights_resnet_1.h5
Epoch 4/100
 - 300s - loss: 0.3140 - acc: 0.9009 - val_loss: 0.3575 - val_acc: 0.8991

Epoch 00004: val_acc did not improve from 0.90198
Epoch 5/100
 - 310s - loss: 0.3193 - acc: 0.9005 - val_loss: 0.2818 - val_acc: 0.9008

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00005: val_acc did not improve from 0.90198
Epoch 6/100
 - 301s - loss: 0.2860 - acc: 0.9042 - val_loss: 0.2


Epoch 00048: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.

Epoch 00048: val_acc did not improve from 0.93242
Epoch 49/100
 - 294s - loss: 0.1412 - acc: 0.9517 - val_loss: 0.2057 - val_acc: 0.9320

Epoch 00049: val_acc did not improve from 0.93242
Epoch 50/100
 - 294s - loss: 0.1410 - acc: 0.9516 - val_loss: 0.2016 - val_acc: 0.9322

Epoch 00050: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.

Epoch 00050: val_acc did not improve from 0.93242
Epoch 51/100
 - 294s - loss: 0.1407 - acc: 0.9516 - val_loss: 0.2031 - val_acc: 0.9320

Epoch 00051: val_acc did not improve from 0.93242
Epoch 52/100
 - 294s - loss: 0.1402 - acc: 0.9519 - val_loss: 0.2046 - val_acc: 0.9319

Epoch 00052: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-11.

Epoch 00052: val_acc did not improve from 0.93242
Epoch 53/100
 - 294s - loss: 0.1404 - acc: 0.9519 - val_loss: 0.2041 - val_acc: 0.9321

Epoch 00053: val_acc did not improve from 0.93242
Epoch 54/100
 - 

In [13]:
model_resnet.load_weights("weights_resnet_1.h5")
model_resnet.evaluate(X_train, y_train, verbose=1)

50000/50000 [==============================] - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 59 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 58 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 57 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 56 - ETA: 55 - ETA: 55 - ETA: 55 - ETA: 55 - ETA: 55 - ETA: 55 - ETA: 55 - ET

[0.1446301045513153, 0.953811996459961]

In [15]:
model_resnet.evaluate(X_test, y_test, verbose=1)

10000/10000 [==============================] - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  

[0.20281124305725098, 0.931239993095398]